In [6]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np;
from tensorflow import keras 
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator

In [7]:
num_classes = 10
(ax, ay), (qx, qy) = cifar10.load_data()
input_shape = (32, 32, 3)
ax = ax.astype('float32'); ax /= 255 #0 a 1
qx = qx.astype('float32'); qx /= 255 #0 a 1
ax -= 0.5; qx -= 0.5 #-0.5 a +0.5
ay = keras.utils.to_categorical(ay, num_classes);
qy = keras.utils.to_categorical(qy, num_classes)

In [8]:
# Fazendo a distorção
test_datagen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    rotation_range=10.,
    fill_mode='nearest', 
    width_shift_range = 0.1, 
    height_shift_range = 0.1
)

In [9]:
model = load_model("resnet1.h5")

In [10]:
tta_steps = 10
predictions = []

from tqdm import tqdm
for i in tqdm(range(tta_steps)):
    preds = model.predict_generator(test_datagen.flow(qx, batch_size=64, shuffle=False), steps = len(qx)/64)
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

np.mean(np.equal(np.argmax(qy, axis=-1), np.argmax(pred, axis=-1)))

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\leona\AppData\Local\Temp\ipykernel_11052\3543264825.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(test_datagen.flow(qx, batch_size=64, shuffle=False), steps = len(qx)/64)
100%|██████████| 10/10 [02:01<00:00, 12.10s/it]


0.931